# 设施选址 Facility Location

等级：初级

## 目的和先决条件

在此示例中，我们将解决设施位置问题，即我们要建立仓库以供应一定数量的超级市场。我们将构建此问题的混合整数编程（MIP）模型，在Gurobi Python界面中实现此模型，并计算最佳解决方案。

该建模示例处于初级阶段，我们假设您了解Python，并且具有一些有关构建数学优化模型的知识。

**注意：** 你可以通过单击 [此处](https://github.com/arvinxx/gurobi-and-mathematical-modeling/archive/master.zip) 下载包含此示例和其他示例的代码。为了正确运行此 Jupyter Notebook，您必须具有 Gurobi 许可证。如果您没有，则可以**商业用户身份**申请 [试用许可证](https://www.gurobi.com/downloads/request-an-evaluation-license/)，或以**学术用户身份**下载 [免费许可证](https://www.gurobi.com/academia/academic-program-and-licenses)。

## 动机

设施选址问题的研究（也称为位置分析）是运筹学和计算几何学的一个分支，与设施的最佳布置有关，以最大程度地降低运输成本，同时考虑避免在房屋附近放置危险材料以及竞争对手设施的位置等因素。

费马-韦伯问题（Fermat-Weber Problem）是17世纪提出的最早的设施选址问题之一。

费马-韦伯问题可以描述如下：

> 给定一个平面中的三个点，找到第四个点，使得其到三个给定点的距离之和最小。

该问题可以解释为设施位置问题的一种形式，其中假设所有目的地的每距离运输成本都相同。

设施选址问题在许多行业都有应用。对于供应链管理和物流，该问题可用于寻找商店、工厂、仓库等的最佳位置。其他应用范围包括公共政策（例如在城市中安置警察）、电信（例如网络中的蜂窝塔），甚至还有粒子物理学（例如，排斥电荷之间的分离距离）。设施选址问题的另一个应用是确定天然气输送设备的位置。最后，设施选址问题可应用于聚类分析。

## 问题描述

英国的一家大型连锁超市需要为其在英格兰北部开设的一系列超市建立仓库。超级市场的位置已经确定，但是仓库的位置尚未确定。

目前已经确定了若干仓库的备选地点，但必须就建立多少个仓库以及在哪些候选地点建立仓库作出决定。

建设多个仓库是有好处的，因为这将减少卡车从仓库到超市的平均行驶距离，从而降低运输成本。但是，建一个仓库有一定的固定成本。

在这个例子中，我们的目标是找到运输成本和建造新仓库成本之间的最佳权衡。


## 解决方法

数学优化（也称为数学编程）是一种声明性方法，其中建模者制定了一个优化问题，该问题捕获了复杂决策问题的关键特征。然后 Gurobi 使用最先进的数学和计算机科学技术来解决这一类数学优化问题。


数学优化模型具有五个组成部分：

* 数据集（Sets）
* 参数(Parameters)
* 决策变量(Decision variables)
* 约束（Constraints）
* 目标函数（Objective function(s)）


我们现在提出了一个设施选址问题的 MIP 模型。

## 模型制定

### 索引和集合

$i \in I$: 超市（或客户）位置的索引和集合。

$j \in J$: 候选仓库（或设施）位置的索引和集合。

### 参数

$f_{j} \in \mathbb{R}^+$: 与建造设施 $j$ 有关的固定成本，其中 $j \in J$。

$d_{i,j} \in \mathbb{R}^+$: 设施  $j \in J$ 和 客户 $i \in I$ 的距离。

$c_{i,j} \in \mathbb{R}^+$: 候选设施 $j \in J$ 和客户 $i \in I$ 之间的运输成本。 我们假设此成本与设施与客户之间的距离成比例。
也就是说， $c_{i,j} = \alpha \cdot d_{i,j}$，其中 $\alpha$ 是每英里的运输成本，将其调整为货车在五年内的平均行程开销。

### 决策变量

$select_{j} \in \{0, 1 \}$: 如果我们在候选位置 $j \in J$ 中建立设施，则此变量等于1，否则为0；

$0 \leq assign_{i,j} \leq 1$: 这个非负连续变量决定了客户 $i\in I$ 从设施 $j \in J$ 收到的货量占总供货量的比例。


### 目标函数

- **总成本**：我们想要把开办和运营这些设施的总成本降到最低。这是建设设施的成本和运输成本的总和。这个总成本衡量了建造一个新设施的成本与五年内总运输成本之间的权衡。

\begin{equation}
\text{Max} \quad Z = \sum_{j \in J} f_{j} \cdot select_{j} + \sum_{j \in J} \sum_{i \in I} c_{i,j} \cdot assign_{i,j}
\tag{0}
\end{equation}

### 约束

- **客户需求**。对于每个客户$i \in I$，确保满足其需求。即，每个客户从每个设施收到的分配比之和必须等于1：

\begin{equation}
\sum_{j \in J} assign_{i,j} = 1 \quad \forall i \in I
\tag{1}
\end{equation}

- **运输**。 我们需要确保让已建设的设施 $j \in J$ 进行运输。

\begin{equation}
assign_{i,j} \leq select_{j} \quad \forall i \in I \quad \forall j \in J
\tag{2}
\end{equation}

## Python 实现


本案例包含 2 个超市和 9 个仓库备选地址。下表提供了各超市的坐标。

| <i></i> | 坐标 |
| --- | --- | 
| 超市 1 | (0,1.5) |
| 超市 2 | (2.5,1.2) |

下表显示了候选仓库的坐标和建造仓库的固定成本（单位：百万英镑）。

| <i></i> | 坐标 | 固定成本 |
| --- | --- |  --- |
| 仓库 1 | (0,0) | 3 |
| 仓库 2 | (0,1) | 2 |
| 仓库 3 | (0,2) | 3 |
| 仓库 4 | (1,0) | 1 |
| 仓库 5 | (1,1) | 3 |
| 仓库 6 | (1,2) | 3 |
| 仓库 7 | (2,0) | 4 |
| 仓库 8 | (2,1) | 3 |
| 仓库 9 | (2,2) | 2 |


每英里的成本为一百万英镑。

## 代码实现

导入 Gurobi 的 Python 模块和相关 Python 库。然后用给定的数据初始化数据结构。

In [1]:
# 官方测试版本 Gurobi v9.0.0 & Python 3.7.0

# 译者测试版本 Gurobi v9.1.0 & Python 3.8.6

from itertools import product
from math import sqrt

import gurobipy as gp
from gurobipy import GRB


# 参数
customers = [(0,1.5), (2.5,1.2)]
facilities = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
setup_cost = [3,2,3,1,3,3,4,3,2]
cost_per_mile = 1

### 预处理

我们定义了一个函数来确定每个设施和客户站点之间的欧式距离。此外，我们还计算了设施选址问题的MIP模型公式所需的关键参数。

In [2]:
# 此函数确定设施和客户站点之间的欧式距离

def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return sqrt(dx*dx + dy*dy)

# 计算MIP模型的关键参数

num_facilities = len(facilities)
num_customers = len(customers)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))

# 计算运输成本

shipping_cost = {(c,f): cost_per_mile*compute_distance(customers[c], facilities[f]) for c, f in cartesian_prod}

### 模型部署

现在我们通过定义决策变量、约束条件和目标函数来确定设备选址问题的MIP模型。然后进行优化计算，Gurobi会找到建造设施的可行解，使总成本最小化。

In [3]:
# MIP 模型

m = gp.Model('facility_location')

select = m.addVars(num_facilities, vtype=GRB.BINARY, name='Select')
assign = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='Assign')

m.addConstrs((assign[(c,f)] <= select[f] for c,f in cartesian_prod), name='Setup2ship')
m.addConstrs((gp.quicksum(assign[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')

m.setObjective(select.prod(setup_cost)+assign.prod(shipping_cost), GRB.MINIMIZE)

m.optimize()

Using license file c:\gurobi\gurobi.lic
Set parameter TokenServer to value SANTOS-SURFACE-
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 20 rows, 27 columns and 54 nonzeros
Model fingerprint: 0x0939f503
Variable types: 18 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 20 rows, 27 columns, 54 nonzeros
Variable types: 18 continuous, 9 integer (9 binary)

Root relaxation: objective 4.723713e+00, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       4.7237129    4.72371  0.00%     -    0s

Explored 0 nodes (15 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 4.72371 

Optima

## 分析

优化模型的结果表明，最小总成本价值为472万英镑。让我们看看达到最佳结果的解决方案。

### 仓库建设计划

该计划决定在哪个厂址的位置建立仓库。

In [4]:
# 显示决策变量的最佳值

for facility in select.keys():
    if (abs(select[facility].x) > 1e-6):
        print(f"\n 建立仓库的地址为： {facility + 1}.")


 Build a warehouse at location 4.


### 运输计划

这个计划决定了从每个工厂发送到每个客户的出货量的百分比。

In [5]:
# 从工厂到客户的运输方案

for customer, facility in assign.keys():
    if (abs(assign[customer, facility].x) > 1e-6):
        print(f"\n 超市 {customer + 1} 从工厂 {facility + 1} 接受 {round(100*assign[customer, facility].x, 2)} % of 的需求")



 Supermarket 1 recieves from Warehouse 4 100.0 % of its demand.

 Supermarket 2 recieves from Warehouse 4 100.0 % of its demand.


##  结论

在此示例中，我们解决了一个设施选址问题，即我们要建立仓库以供应大量的超市，同时最大程度地减少建造仓库的固定总成本和从仓库到超市的可变运输总成本。我们学习了如何将问题表达为MIP模型。此外，我们还学习了如何实现 MIP 模型并使用 Gurobi 对其求解。


##  参考资料

[1] Laporte, Gilbert, Stefan Nickel, and Saldanha da Gama, Francisco. Location Science. Springer, 2015.

Copyright © 2020 Gurobi Optimization, LLC

翻译 By Arvin Xu